<a href="https://colab.research.google.com/github/luismiguelcasadodiaz/IBM_SkillsBuild_IA_325/blob/main/IA_325_py_cod_ex_20_s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predecir resultados en partidas multijugador
## 🧠 Objetivo

En este ejercicio, aplicarás tus conocimientos de regresión logística para construir un modelo capaz de predecir si un jugador ganó o perdió una partida, a partir de sus estadísticas individuales.



## 📋 Descripción del problema

Tienes que construir un modelo predictivo que, a partir de las estadísticas de un jugador en una partida, determine si ganó o no. Para ello, deberás:

+ Crear datos sintéticos que representen partidas ficticias de jugadores.
+ Entrenar un modelo de regresión logística con esos datos.
+ Implementar una función que prediga el resultado (ganar o no) para un nuevo jugador.



## 📦 Paso 1: Definir una clase PlayerMatchData para representar una **partida**

  + Atributos:
    + kills: número de enemigos eliminados
    + deaths: número de veces que el jugador ha muerto
    + assists: asistencias realizadas
    + damage_dealt: daño total infligido
    + damage_received: daño total recibido
    + healing_done: curación realizada
    + objective_time: tiempo (en segundos) que el jugador estuvo capturando objetivos
    + won: 1 si el jugador ganó la partida, 0 si perdió

  + Métodos:
    + .to_dict() que devuelva los datos como un diccionario (sin la variable won, opcionalmente).


## 📦 Paso 2: Generar datos sintéticos con NumPy

Crea una función llamada generate_synthetic_data que genere un conjunto de datos de entrenamiento simulando partidas de videojuegos. Para ello:

Utiliza la librería numpy para generar los valores numéricos.

Cada instancia representará el desempeño de un jugador en una partida.

La función debe devolver una lista de objetos PlayerMatchData (ya definida previamente).

Implementa la siguiente lógica para cada jugador:

Reglas para los datos:

+ kills: número de enemigos eliminados, generado con una distribución de Poisson con media 5.
+ deaths: número de veces que el jugador ha muerto, distribución de Poisson con media 3.
+ assists: asistencias realizadas, distribución de Poisson con media 2.
+ damage_dealt: daño infligido, calculado como kills * 300 + ruido aleatorio normal.
+ damage_received = deaths * 400 + np.random.normal(0, 100)
damage_received: daño recibido, como deaths * 400 + ruido aleatorio normal.
+ healing_done: cantidad de curación, valor aleatorio entero entre 0 y 300.
+ objective_time: tiempo (en segundos) controlando objetivos, valor aleatorio entre 0 y 120.
+ won: el jugador se considera que ganó la partida si hizo más daño del que recibió y tuvo más kills que muertes.

Para un aleatorio con distribución de poisson de media 5 usar:
```python
kills = np.random.poisson(5)
```

Para un ruido aleatorio normal usar:
```python
ruido = np.random.normal(0, 100)
```


### 🧠 Tu función debe seguir esta estructura:
```python
import numpy as np

def generate_synthetic_data(n=100):
    data = []
    for _ in range(n):
        # Genera cada variable siguiendo las instrucciones dadas
        # Crea un objeto PlayerMatchData con estos valores
        # Añádelo a la lista de datos

    return data

```

##🧪 Paso 3: Crear y entrenar el modelo

Crea una clase VictoryPredictor que entrene un modelo de regresión logística con los datos sintéticos. Esta clase debe tener:
  + Métodos:
    + Un método train(data) para entrenar el modelo.
    + Un método predict(player: PlayerMatchData) que devuelva 1 si predice victoria, 0 si derrota.



## 📌 Ejemplo de uso
```python
# Crear datos de entrenamiento
training_data = generate_synthetic_data(150)

# Entrenar modelo
predictor = VictoryPredictor()
predictor.train(training_data)

# Crear jugador de prueba
test_player = PlayerMatchData(8, 2, 3, 2400, 800, 120, 90, None)

# Predecir si ganará
prediction = predictor.predict(test_player)
print(f"¿El jugador ganará? {'Sí' if prediction == 1 else 'No'}")
```

## Salida esperada

¿El jugador ganará? Sí

### Importación de librerias

In [68]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model  import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [69]:
class PlayerMatchData:
    """Almacena datos de una partida de jugador con estadísticas clave.

    Incluye kills, deaths, assists, daño infligido/recibido, curación,
    tiempo en objetivo y si la partida fue ganada o no.
    """
    def __init__(self, kills, deaths, assists, damage_dealt,
                 damage_received, healing_done, objective_time, won=None):
        """Inicializa los datos de la partida del jugador.

        Args:
            kills (int): Número de asesinatos realizados.
            deaths (int): Número de veces que el jugador murió.
            assists (int): Número de asistencias realizadas.
            damage_dealt (float): Cantidad de daño infligido.
            damage_received (float): Cantidad de daño recibido.
            healing_done (float): Cantidad de curación realizada.
            objective_time (float): Tiempo en segundos controlando objetivos.
            won (int, optional): 1 si la partida se ganó, 0 si se perdió,
                None si no se especifica. Defaults to None.

        Raises:
            ValueError: Si los tipos o valores de los argumentos no son válidos.
        """
        if not all(isinstance(arg, int) and arg >= 0
                   for arg in [kills, deaths, assists]):
            raise ValueError("Kills, deaths, and assists must be non-negative integers.")
        if not all(isinstance(arg, (int, float)) and arg >= 0.0
                   for arg in [damage_dealt, damage_received, healing_done,
                               objective_time]):
            raise ValueError("damage_dealt, damage_received, healing_done, and "
                             "objective_time must be non-negative numbers.")
        if won is not None and won not in [0, 1]:
            raise ValueError("WON must be 0 or 1 or None.")

        self.kills = kills
        self.deaths = deaths
        self.assists = assists
        self.damage_dealt = damage_dealt
        self.damage_received = damage_received
        self.healing_done = healing_done
        self.objective_time = objective_time
        self.won = won

    def to_dict(self):
        """Devuelve un diccionario con los atributos de la instancia."""
        return vars(self)

    def __str__(self):
        """Devuelve una representación en cadena del objeto."""
        return str(self.to_dict())



##### pruebas a la clase Partida

In [70]:
test_player = PlayerMatchData(8, 2, 3, 2400, 800, 120, 90, None)
print(test_player.to_dict())
print(test_player)

{'kills': 8, 'deaths': 2, 'assists': 3, 'damage_dealt': 2400, 'damage_received': 800, 'healing_done': 120, 'objective_time': 90, 'won': None}
{'kills': 8, 'deaths': 2, 'assists': 3, 'damage_dealt': 2400, 'damage_received': 800, 'healing_done': 120, 'objective_time': 90, 'won': None}


### Creación de la función generate_synthetic_data

In [90]:
def generate_synthetic_data(n=100):
    """Genera datos sintéticos de partidas de jugadores.

    Crea una lista de objetos PlayerMatchData con valores aleatorios
    para simular estadísticas de juego.

    Args:
        n (int, optional): Número de partidas sintéticas a generar.
            Defaults to 100.

    Returns:
        list: Una lista de objetos PlayerMatchData generados
              aleatoriamente.
    """
    data = []
    for _ in range(n):
        # Genera cada variable siguiendo las instrucciones dadas
        kills           = np.random.poisson(5)
        deaths          = np.random.poisson(3)
        assists         = np.random.poisson(2)
        invalido = True
        while invalido:
          damage_dealt    = 300 * kills + np.random.normal(0, 100)
          invalido = (damage_dealt < 0)
        invalido = True
        while invalido:
          damage_received = 400 * deaths + np.random.normal(0, 100)
          invalido = (damage_received < 0)
        healing_done    = np.random.randint(0, 300)
        objective_time  = np.random.randint(0, 120)
        won             = 1 if (damage_dealt > damage_received ) else 0
        # Crea un objeto PlayerMatchData con estos valores
        jugador_nuevo = PlayerMatchData( kills, deaths, assists, damage_dealt, \
              damage_received, healing_done, objective_time, won)
        data.append(jugador_nuevo )# Añádelo a la lista de datos

    return data

#### Tests a la función generate_syntehtic_data

In [72]:
datos = generate_synthetic_data(2)
print(type(datos))
for dato in datos:
  print(type(dato), dato)

<class 'list'>
<class '__main__.PlayerMatchData'> {'kills': 6, 'deaths': 3, 'assists': 5, 'damage_dealt': 1801.4461760088561, 'damage_received': 1239.6905024762088, 'healing_done': 260, 'objective_time': 19, 'won': 1}
<class '__main__.PlayerMatchData'> {'kills': 4, 'deaths': 3, 'assists': 1, 'damage_dealt': 1182.5349440844661, 'damage_received': 1022.4105783089286, 'healing_done': 222, 'objective_time': 113, 'won': 1}


### Creación de la clase VictoryPredictor

In [112]:
class VictoryPredictor:
    """Predice la probabilidad de victoria en una partida.

    Utiliza regresión logística para clasificar si un jugador ganará
    basándose en sus estadísticas de partida.
    """
    def __init__(self):
        """Inicializa el predictor con un modelo y un escalador."""
        self.model = LogisticRegression()
        self.scaler = StandardScaler()

    def train(self, datos):
        """Entrena el modelo con los datos de partidas proporcionados.

        Args:
            datos (list): Una lista de objetos PlayerMatchData
                utilizados para el entrenamiento.
        """
        X = [[j.kills, j.deaths, j.assists, j.damage_dealt,
              j.damage_received, j.healing_done, j.objective_time]
             for j in datos]
        y = [j.won for j in datos]
        self.scaler.fit(X)
        X_scaled = self.scaler.transform(X)
        self.model.fit(X_scaled, y)

    def predict(self, j: PlayerMatchData):
        """Predice si un jugador ganará la partida.

        Args:
            j (PlayerMatchData): Un objeto PlayerMatchData con las
                estadísticas de la partida a predecir.

        Returns:
            ndarray: Un array numpy con la predicción (0 o 1).
        """
        X = [[j.kills, j.deaths, j.assists, j.damage_dealt,
              j.damage_received, j.healing_done, j.objective_time]]
        X_scaled = self.scaler.transform(X)
        return self.model.predict(X_scaled.reshape(1, -1))

In [111]:
datos = generate_synthetic_data(40)
vp = VictoryPredictor()
vp.train(datos)
print("coeficioentes :" ,vp.model.coef_)
print("intercep :",vp.model.intercept_)
test_player = PlayerMatchData(8, 2, 3, 2400, 800, 120, 90, None)
predicción = vp.predict(test_player)
print(f"¿El jugador ganará? {'Sí' if predicción == 1 else 'No'}")

coeficioentes : [[ 1.01690032 -1.28160714 -0.28413591  1.20492955 -1.35648008 -0.29583005
  -0.2629383 ]]
intercep : [1.734082]
¿El jugador ganará? Sí
